# PART 1: Initial Analysis

In [1]:
ctext = 'lkioogtkjimqhohiqljqkjumffqkmpjh' #text to decrypt

In [2]:
from collections import Counter
count = Counter(ctext)
count, len(count)

(Counter({'l': 2,
          'k': 4,
          'i': 3,
          'o': 3,
          'g': 1,
          't': 1,
          'j': 4,
          'm': 3,
          'q': 4,
          'h': 3,
          'u': 1,
          'f': 2,
          'p': 1}),
 13)

In [3]:
sorted(count.keys())

['f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'o', 'p', 'q', 't', 'u']

In [4]:
#generate the 16 alphabets present from f-u 
alpha = [chr(x) for x in range(ord('f'), ord('u') + 1)]
alpha, len(alpha) 

(['f',
  'g',
  'h',
  'i',
  'j',
  'k',
  'l',
  'm',
  'n',
  'o',
  'p',
  'q',
  'r',
  's',
  't',
  'u'],
 16)

In [5]:
#generate mapping of alphabets to binary numbers
alphamap = {}
for i in range(len(alpha)):
    alphamap[alpha[i]] = bin(i)[2:].zfill(4)
alphamap    

{'f': '0000',
 'g': '0001',
 'h': '0010',
 'i': '0011',
 'j': '0100',
 'k': '0101',
 'l': '0110',
 'm': '0111',
 'n': '1000',
 'o': '1001',
 'p': '1010',
 'q': '1011',
 'r': '1100',
 's': '1101',
 't': '1110',
 'u': '1111'}

To break 6-round DES, we need 4-round characteristic. Our 4 round characteristic is 
(405C0000,04000000,1/4 ; 04000000,00540000,5/128 ; 00540000, 00000000,1; 00000000, 00540000, 5/128, 00540000,04000000) 

In [6]:
#Overall probability of characteristic
p = (1/4) * (5/128) * (1) * (5/128)
p

0.0003814697265625

In [7]:
#the input xor after initial permutation
input_2 = bin(int('405C000004000000', 16))[2:].zfill(64)
input_2

'0100000001011100000000000000000000000100000000000000000000000000'

In [8]:
#INITIAL PERMUTATION (IP) 
#8x8
IP = [
  58,50,42, 34,26,18,10,2,
  60,52,44,36,28,20,12,4,
  62,54, 46, 38, 30, 22, 14,6,
  64, 56, 48, 40,32,24, 16, 8,
  57, 49, 41, 33,25,17, 9,1,
  59, 51,43,35,27,19,11,3,
  61,53,45,37,29,21,13, 5,
  63,55, 47,39,31,23,15,7
    ]

#FINAL PERMUTATION (FP)
#8x8
FP = [
    40,8,48,16,56,24,64,32,
    39,7,47,15,55,23,63,31,
    38,6,46,14,54,22,62,30,
    37,5,45,13,53,21,61,29,
    36,4,44,12,52,20,60,28,
    35,3,43,11,51,19,59,27,
    34,2,42,10,50,18,58,26,
    33,1,41,9,49,17,57,25,  
]

In [9]:
#mixing of input_2 according to FP (reverse of IP)
input_1 = ''
for i in range(len(input_2)):
    input_1 += input_2[FP[i]-1]
input_1    

'0000000000000000100100000001000000010000000000000101000000000000'

In [10]:
hex(int(input_1, 2))[2:].zfill(16)

'0000901010005000'

# PART 2 : Generating random plaintext pairs and finding corresponding ciphertext pairs using script

In [ ]:
#using the script and commands mentioned in analysis.

#  PART 3: Get input XOR to S-boxes in round 6. (beta xor beta')

In [11]:
n1_list = [] #number1 list of outputs (after rev FP (IP) and swapping) for input1
n2_list = [] #number2 list of outputs (after rev FP (IP) and swapping) for input2
#input1 xor input2 = 0000 9010 1000 5000
with open('ciphertext.txt', 'r') as fp:
    flag = 0
    for line in fp:
        if flag == 0:
            temp = ''
            fop = line.strip() #final output
            for j in range(64):
                temp += fop[IP[j]-1]
            temp2 = ''
            temp2 += temp[32:]
            temp2 += temp[:32]
            n1_list.append(temp2)
            flag = 1
        else:
            temp = ''
            fop = line.strip() #final output
            for j in range(64):
                temp += fop[IP[j]-1]
            temp2 = ''
            temp2 += temp[32:]
            temp2 += temp[:32]
            n2_list.append(temp2)
            flag = 0

In [12]:
len(n1_list), len(n2_list)

(100000, 100000)

In [13]:
n1_l4_list = []
n1_r4_list = []
n2_l4_list = []
n2_r4_list = []

for i in range(len(n1_list)):
    n1_l4_list.append(n1_list[i][:32])
    n2_l4_list.append(n2_list[i][:32])
    
    n1_r4_list.append(n1_list[i][32:])
    n2_r4_list.append(n2_list[i][32:])

In [14]:
#Expansion table (8x6)
E = [
  32, 1, 2, 3, 4, 5,
  4, 5,6, 7, 8, 9,
  8, 9, 10, 11, 12, 13,
  12, 13, 14, 15, 16, 17,
  16, 17, 18, 19, 20, 21,
  20, 21, 22, 23, 24, 25,
  24, 25, 26, 27, 28, 29,
  28, 29, 30, 31, 32, 1
]

In [15]:
alpha1_list = []
alpha2_list = []
#alpha1 and alpha2 are values just after expansion in round 6

for i in range(len(n1_list)):
    temp = ''
    for j in range(48):
        temp += n1_l4_list[i][E[j]-1]
    alpha1_list.append(temp)
    
    temp = ''
    for j in range(48):
        temp += n2_l4_list[i][E[j]-1]
    alpha2_list.append(temp)

In [16]:
with open('alpha1.txt', 'w') as fp:
    for i in alpha1_list:
        fp.write(i + '\n')

In [17]:
#find alpha1 xor alpha2 = beta1 xor beta2
#beta1 and beta2 are inputs to the S-boxes in round 6.

betaxor_list = [] #xor values for beta1 and beta2 

for i in range(len(alpha1_list)):
    betaxor_list.append(bin(int(alpha1_list[i], 2) ^ int(alpha2_list[i], 2))[2:].zfill(48))      

In [18]:
with open('betaxor.txt', 'w') as fp:
    for i in betaxor_list:
        fp.write(i + '\n')

# PART 4: Get output XOR after S-boxes in round 6 (with probability p) (gamma xor gamma')

In [19]:
INV_P = [
    9, 17, 23, 31,
    13, 28, 2, 18,
    24, 16, 30, 6,
    26, 20, 10, 1,
    8, 14, 25, 3,
    4, 29, 11, 19,
    32, 12, 22, 7,
    5, 27, 15, 21,
]

In [20]:
#n1_r4_list and n2_r4_list have r4 values of round 6. XOR them: (r4 xor r4') xor 0400 0000 (which is given with probability p)
gammaxor_list = []

for i in range(len(n1_r4_list)):
    pop_xor = bin(int(n1_r4_list[i], 2) ^ int(n2_r4_list[i], 2) ^ int('04000000', 16))[2:].zfill(32) #P-box output xor
    temp = ''    
    for j in range(32):
        temp += pop_xor[INV_P[j]-1]
    gammaxor_list.append(temp)    

In [21]:
with open('gammaxor.txt', 'w') as fp:
    for i in gammaxor_list:
        fp.write(i + '\n')

# PART 5: Find all possible pairs (beta,beta'), the set Xi and the set Ki

In [22]:
S = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3 , 10, 6, 12, 5, 9, 0, 7,
    0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8,
    4, 1 , 14, 8, 13, 6, 2, 11, 15, 12, 9, 7,3, 10, 5, 0,
    15, 12, 8,2,4, 9, 1,7 , 5, 11, 3, 14, 10, 0, 6, 13] ,

    [15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0,5, 10,
    3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5,
    0, 14, 7, 11, 10, 4, 13, 1, 5, 8,12, 6, 9, 3, 2, 15,
    13, 8, 10, 1, 3, 15, 4, 2,11,6, 7, 12, 0,5, 14, 9],

    [10, 0, 9,14,6,3,15,5, 1, 13, 12, 7, 11, 4,2,8,
    13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1,
    13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12,5, 10, 14, 7,
    1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12],

    [7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15,
    13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9,
    10, 6, 9, 0, 12, 11, 7, 13, 15, 1 , 3, 14, 5, 2, 8, 4,
    3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14],

    [2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9,
    14, 11,2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6,
    4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14,
    11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3],

    [12, 1, 10, 15, 9, 2, 6,8, 0, 13, 3, 4, 14, 7, 5, 11,
    10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8,
    9, 14, 15, 5, 2,8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6,
    4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13],

    [4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1,
    13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6,
    1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2,
    6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12],

    [13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12,7,
    1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2,
    7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8,
    2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]
  ]
  

In [23]:
rows, cols = (8, 64) 
key_count = [[0 for i in range(cols)] for j in range(rows)] 

In [24]:
#we know the values (beta1 xor beta2): 48bits, (gamma1 xor gamma2):32bits
#do the next step for 6-bit pairs, representing each S-box 
#find the pairs (beta, beta') such that beta xor beta' = beta1 xor beta2 AND s(beta) xor s(beta') = gamma1 xor gamma2

for ind in range(100000):
    alpha = alpha1_list[ind]
    beta_xor = betaxor_list[ind]
    gamma_xor = gammaxor_list[ind]

    for box in range(8):
        temp_a = alpha[6*box:6*box+6]
        temp_g = gamma_xor[4*box:4*box+4]
        temp_b = beta_xor[6*box:6*box+6]
    
        for j in range(64):
            b1 = bin(j)[2:].zfill(6)
            b2 = bin(j ^ int(temp_b, 2))[2:].zfill(6)
            
            row = int(b1[0] + b1[-1], 2)
            col = int(b1[1:-1], 2)
            gamma = bin(S[box][row*16 + col])[2:].zfill(6)

            row = int(b2[0] + b2[-1], 2)
            col = int(b2[1:-1], 2)
            gammaprime = bin(S[box][row*16 + col])[2:].zfill(6)

            if ((int(gamma, 2) ^ int(gammaprime, 2)) == int(temp_g, 2)): 
                key_count[box][int(b1, 2) ^ int(temp_a, 2)] += 1

In [25]:
for i in range(8):
    print(max(key_count[i]), sum(key_count[i])/len(key_count[i])) 

8311 6448.25
8733 6482.5
6476 6254.8125
6454 6289.09375
9393 6569.6875
8440 6483.53125
8426 6482.9375
8579 6567.1875


In [26]:
key = [-1] * 8
for i in range(8):
    max_ind, max_val = -1, 0
    for j in range(64):
        if key_count[i][j] > max_val:
            max_val = key_count[i][j]
            max_ind = j
    key[i] = max_ind        

In [27]:
key[2], key[3] = -1, -1
key

[45, 59, -1, -1, 44, 57, 31, 52]

# PART 6: Finding the key (Brute Force and Decryption)

In [28]:
#k6 is found using the program deskey6.cpp

In [29]:
k6 =[24, 27, 21, 6, 11, 15, 
    13, 10, 25, 16, 3, 20, 
    5, 1, 22, 14, 8, 18, 
    26, 17, 9, 2, 23, 12, 
    51, 34, 41, 47, 29, 37,
    40, 50, 33, 55, 43, 30, 
    54, 31, 49, 38, 44, 35, 
    56, 52, 32, 46, 39, 42 ]

In [30]:
#mapping the key according to the above permutation
#k6,1 bits are mapped to bit positions 24, 27, 21, 6, 11, 15 and so on for every k6,i
keyfinal = ['x'] * 56

for i in range(8):
    if (i == 2 or i == 3):
        continue
    bkey = bin(key[i])[2:].zfill(6)
    #print(bkey)
    ind = 0
    for j in range(i*6, i*6 + 6):
        keyfinal[k6[j]-1] = bkey[ind]
        ind += 1
keyfinal
#fkey is the final key with some bits missing
fkey = ''
for i in keyfinal:
    fkey += i
fkey    

'xx1xx1xxx10x1x10xxx11xx11x0x0110101x01011001x11x1111x001'

In [31]:
with open('possiblekeys.txt', 'w') as fp:
    for i in range(pow(2, 20)):
        bin20 = bin(i)[2:].zfill(20)
        #print(bin20)
        ind = 0
        xkey = keyfinal.copy()
        for j in range(56):
            if (keyfinal[j] == 'x'):
                xkey[j] = bin20[ind]
                ind += 1
        ffkey = ''
        for j in range(len(xkey)):
            ffkey += xkey[j]
        fp.write(ffkey + '\n')    

In [32]:
ip = 'ffffffffffffffff'
op = 'qrnpthmqggoljtqf'
ipval8 = [0, 0, 0, 0, 0, 0, 0, 0]
opval8 = []

for i in range(0, len(op), 2):
    opval8.append(int(alphamap[op[i]] + alphamap[op[i+1]], 2))
    
opval8    

[188, 138, 226, 123, 17, 150, 78, 176]

In [33]:
#find key using des_bruteforce.cpp

In [34]:
key = '01101110010111100111101110000110101001011001011011111001'

In [35]:
ctext8 = []
for i in range(0, len(ctext), 2):
    ctext8.append(int(alphamap[ctext[i]] + alphamap[ctext[i+1]], 2))
ctext, ctext8

('lkioogtkjimqhohiqljqkjumffqkmpjh',
 [101, 57, 145, 229, 67, 123, 41, 35, 182, 75, 84, 247, 0, 181, 122, 66])

In [36]:
#run the program des_decrypt.cpp twice to get the answer

In [37]:
answer = [116, 114, 115, 121, 100, 119, 102, 112, 105, 113, 48, 48, 48, 48, 48, 48]

In [38]:
#convert from ASCII code to character to get password
for i in answer:
    print(chr(i), end="")

trsydwfpiq000000